In [104]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup as BS
from konlpy.tag import Mecab
import pandas as pd
from collections import Counter

In [2]:
mecab = Mecab()

In [3]:
today = datetime.today().strftime("%Y%m%d")

In [4]:
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

In [5]:
def get_news_links(page):
    url = base_url + "&page=" + str(page)
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])
    
    return links

def get_news_content(url):
    response = requests.get(url)
    soup = BS(response.text, "html.parser")
    
    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article',{'id':'dic_area'})
    
    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

In [6]:
# 크롤링 시작
news_links = []
page = 1

while True:
    links = get_news_links(page)
    if not links or any(link in news_links for link in links):
        break
    news_links.extend(links)
    page += 1

In [8]:
len(news_links)

13752

In [9]:
# 뉴스 기사 내용 크롤링
news_contents = []
for link in news_links:
    try:
        title, content = get_news_content(link)
        news_contents.append((title, content))
    except Exception as e:
        print(f"Failed to get content from {link}: {e}")

In [11]:
#중복값 제거
set(news_contents)

{("대전 서구 도마·변동 리딩단지…'힐스테이트 가장더퍼스트' 분양 중",
  '현대건설 컨소시엄이 대전시 서구 가장동 일대에 공급하는 \'힐스테이트 가장더퍼스트\'가 선착순 동·호수 지정 계약을 진행 중이다.ⓒ현대건설[데일리안 = 배수람 기자] 현대건설 컨소시엄이 대전시 서구 가장동 일대에 공급하는 \'힐스테이트 가장더퍼스트\'가 선착순 동·호수 지정 계약을 진행 중이다.단지는 대전 서구 가장동 38-1번지 일원에 위치하며, 지하 2층~지상 38층, 15개동, 전용 59~84㎡, 총 1779가구 규모의 대단지로 조성된다. 입주는 2027년 6월 예정이며, 전매제한 기간이 6개월이어서 입주 전 전매가 가능한 것이 특징이다.선착순 동·호수 지정 계약은 견본주택에 방문 후 진행 가능하다.오는 29일부터 진행되는 계약에 한해 1차 계약금 500만원, 계약금 5% 혜택을 제공해 수요자들의 자금부담을 낮췄으며, 향후 계약조건 변경 시 기존 계약자를 포함해 소급적용하는 계약 안심 보장플랜도 시행한다. 단, 6월 28일까지 계약자에 한해서는 계약금 5% 혜택이 적용되지 않는다.단지는 일대를 대표할 리딩 아파트, 즉 대장주로 각광받는 단지다. 단지가 들어서는 도마·변동 재정비 촉진지구는 대전시 서구 도마동과 가장동 일원에 약 2만5000여가구 조성을 목표로 대규모 정비사업이 진행 중이다. 향후 주거환경 발전 가능성이 높은 신흥 주거지다.도마·변동 재정비 촉진지구 입지로 각종 교통호재도 갖추고 있다. 단지에서 도보거리로는 용문역이 있고 인근에 KTX서대전역이 위치해 있어 교통편의성이 뛰어나다. 특히 2026년 개통을 목표로 충청권 광역철도의 1단계(계룡~신탄진 구간) 사업이 착공에 들어간 가운데, 지구 내에 충청권 광역철도 1단계에 해당하는 도마역 신설이 예정돼 있다.대전 도시철도 2호선 트램은 대전광역시를 순환하는 38.1㎞ 노선으로 올해 중 착공 예정이며, 2028년 개통을 목표로 사업이 추진 중이다. 인근 신설될 용두역(올해 하반기 착공 계획)은 충청권 광역철도는 물론,

In [12]:
len(set(news_contents))

6811

In [84]:

# Create a DataFrame to store titles and nouns
df = pd.DataFrame(columns=["Title", "Nouns"])

# Text analysis using Mecab and store in DataFrame
for i, (title, content) in enumerate(set(news_contents), start=1):
    if content:
        nouns = mecab.nouns(content)
        new_row = pd.DataFrame({"Title": [title], "Nouns": [nouns]})
        df = pd.concat([df, new_row], ignore_index=True)        

# Display the DataFrame
print(df)

                                          Title  \
0          대전 서구 도마·변동 리딩단지…'힐스테이트 가장더퍼스트' 분양 중   
1              "지갑 분실해도 위치확인"…삼성전자, IoT 신용카드 출시   
2               서울 아파트 매매가 14주 연속↑…전세값은 58주째 상승   
3              어그, 스타필드 코엑스몰에 단독 매장 오픈…서울 최대 규모   
4             '자금 마른' 오피스·물류센터 시장에…해외 투자자들 '눈독'   
...                                         ...   
6795  배송비 혜택 강화하는 컬리 "멤버십 고객, 2만원 이상 구매하면 무료배송"   
6796      “기술특례기업은 따로 심사”...전담팀 꾸려 상장예심 빨라지게 한다   
6797                  가파른 저출산 고령화 30년뒤 청년인구 반토막   
6798      KB국민, 은행권 최고 출산장려금 '최대 2천만원'…육아 지원 확대   
6799                              [인사] 축산물품질평가원   

                                                  Nouns  
0     [현대건설, 컨소시엄, 대전시, 서구, 가장동, 일대, 공급, 힐, 스테이트, 퍼스...  
1     [국민카드, 협업, 전력, 블루투스, 기술, 스마트, 싱, 드, 활용, 신용, 카드...  
2     [지방, 매매, 전세가, 하락세, 경기도, 광주시, 남한산성, 서울, 아파트, 모습...  
3     [서울, 뉴시스, 스타, 필드, 코엑스, 점, 내부, 사진, 신세계, 인터, 내셔날...  
4     [해외, 운용, 사, 공매, 시장, 강남역, 초, 역세, 땅, 서울역, 타워, 김포...  
...                                    

In [36]:
df.to_csv("article_to_Nouns.csv", index=False, encoding='utf-8-sig')

In [105]:
Counter(df.loc[1,'Nouns'])

Counter({'카드': 16,
         '신용': 13,
         '수': 8,
         '스마트폰': 7,
         '삼성전자': 7,
         '스마트': 6,
         '싱': 6,
         '위치': 6,
         '확인': 6,
         '드': 5,
         '국민카드': 4,
         '협업': 3,
         '기술': 3,
         '출시': 3,
         '일': 3,
         '갤럭시': 3,
         '연결': 3,
         '전력': 2,
         '블루투스': 2,
         '활용': 2,
         '서비스': 2,
         '장소': 2,
         '간': 2,
         '적용': 2,
         '가능': 1,
         '사물': 1,
         '인터넷': 1,
         '기반': 1,
         '파인': 1,
         '태블릿': 1,
         '웨어': 1,
         '러블': 1,
         '태그': 1,
         '등': 1,
         '다양': 1,
         '종류': 1,
         '삼성': 1,
         '기기': 1,
         '특화': 1,
         '이': 1,
         '지갑': 1,
         '분실': 1,
         '보관': 1,
         '경우': 1,
         '국내외': 1,
         '통신': 1,
         '해제': 1,
         '마지막': 1,
         '최근': 1,
         '기록': 1,
         '모델': 1,
         '소개': 1,
         '사진': 1,
         '제공': 1,
         '이용해': 1,

In [96]:
def create_word_count_df(nouns_column):
    # Counter 객체 생성
    counter = Counter()
    
    # Nouns 컬럼의 각 행에 있는 단어들을 업데이트
    for nouns in nouns_column:
        # 각 행의 명사 리스트를 문자열로 읽어들이고, 쉼표와 공백을 제거한 후 리스트로 변환
        # nouns_list = nouns.strip('[]').replace("'", "").split(', ')
        nouns_list = eval(nouns)
        counter.update(nouns_list)
    
    # 중복된 값과 개수를 저장할 리스트
    data = []
    
    # Counter 객체에서 중복된 값과 개수를 추출하여 리스트에 저장
    for item, count in counter.items():
        data.append({'단어': item, '개수': count})
    
    # 데이터프레임 생성
    word_count_df = pd.DataFrame(data)
    
    return word_count_df

In [97]:
create_word_count_df(df['Nouns'])

TypeError: eval() arg 1 must be a string, bytes or code object

In [60]:
# 기존 방식으로 Nouns 컬럼의 각 행에 있는 단어들의 총합 계산
counter_old = Counter()

for nouns in df['Nouns']:
    # 각 행의 명사 리스트를 문자열로 읽어들이고, 쉼표와 공백을 제거한 후 리스트로 변환
    # nouns_list = nouns.strip('[]').replace("'", "").split(', ')
    counter_old.update(nouns)

In [63]:
counter_old

Counter({'등': 17956,
         '일': 14642,
         '것': 11874,
         '년': 11677,
         '수': 11371,
         '원': 10976,
         '기업': 8395,
         '만': 8151,
         '월': 8143,
         '억': 6769,
         '서울': 5704,
         '금융': 5670,
         '시장': 5512,
         '사업': 5394,
         '투자': 5210,
         '중': 5167,
         '제공': 5135,
         '지원': 5117,
         '개': 5070,
         '말': 4915,
         '한국': 4679,
         '이': 4646,
         '기술': 4510,
         '기자': 4198,
         '서비스': 4071,
         '산업': 3988,
         '개발': 3847,
         '이번': 3839,
         '국내': 3812,
         '미국': 3800,
         '가능': 3625,
         '글로벌': 3584,
         '경제': 3453,
         '계획': 3449,
         '대표': 3361,
         '지역': 3355,
         '제품': 3287,
         '올해': 3277,
         '명': 3223,
         '그룹': 3202,
         '달러': 3176,
         '고객': 2999,
         '조': 2957,
         '지난해': 2900,
         '사진': 2897,
         '진행': 2872,
         '공급': 2789,
         '규모': 2766

In [35]:
duplicated_df.sort_values(by="개수",ascending=False)

,값,개수
8,보험,9
12,금융,8
7,생명,7
11,우리,7
19,매각,5
9,인수,5
22,손해,4
0,은행,3
26,사,3
23,입찰,3


In [30]:
import aiohttp
import asyncio
from collections import Counter
from datetime import datetime
from bs4 import BeautifulSoup as BS
from konlpy.tag import Mecab
import pandas as pd
import nest_asyncio

# Mecab 객체 생성
mecab = Mecab()

# nest_asyncio 적용
nest_asyncio.apply()

today = datetime.today().strftime("%Y%m%d")
base_url = "https://news.naver.com/main/list.naver?mode=LSD&mid=shm&sid1=101&date=" + today

async def fetch(session, url, retries=3):
    for i in range(retries):
        try:
            async with session.get(url, timeout=aiohttp.ClientTimeout(total=10)) as response:
                return await response.text()
        except (aiohttp.ClientOSError, aiohttp.ClientResponseError, aiohttp.ServerTimeoutError) as e:
            if i < retries - 1:
                print(f"Retrying ({i + 1}/{retries})...")
                await asyncio.sleep(2)
            else:
                print(f"Failed to fetch {url} after {retries} retries")
                return None

async def get_news_links(session, page):
    url = base_url + "&page=" + str(page)
    response_text = await fetch(session, url)
    if response_text is None:
        return []
    soup = BS(response_text, "html.parser")

    links = []
    for a in soup.select("ul.type06_headline li dl dt a"):
        links.append(a["href"])
    for a in soup.select("ul.type06 li dl dt a"):
        links.append(a["href"])

    return links

async def get_news_content(session, url):
    response_text = await fetch(session, url)
    if response_text is None:
        return None, None
    soup = BS(response_text, "html.parser")

    title_tag = soup.select_one("h2.media_end_head_headline")
    content_tag = soup.find('article', {'id': 'dic_area'})

    if title_tag and content_tag:
        title = title_tag.get_text().strip()
        content = content_tag.get_text().strip()
        return title, content
    else:
        return None, None

async def main():
    async with aiohttp.ClientSession() as session:
        news_links = []
        page = 1

        while True:
            links = await get_news_links(session, page)
            if not links or any(link in news_links for link in links):
                break
            news_links.extend(links)
            page += 1

        tasks = [get_news_content(session, link) for link in news_links]
        news_contents = await asyncio.gather(*tasks)

        # 중복값 제거
        news_contents = list(set(news_contents))

        # Create a DataFrame to store titles and nouns
        df = pd.DataFrame(columns=["Title", "Nouns"])

        # Text analysis using Mecab and store in DataFrame
        for i, (title, content) in enumerate(news_contents, start=1):
            if content:
                nouns = mecab.nouns(content)
                new_row = pd.DataFrame({"Title": [title], "Nouns": [nouns]})
                df = pd.concat([df, new_row], ignore_index=True)

        # Save the DataFrame to a CSV file
        df.to_csv("news_nouns.csv", index=False, encoding='utf-8-sig')

        # Display the DataFrame
        print(df)
        if not df.empty:
            test = df.loc[0, 'Nouns']

            # Counter 객체 생성
            counter = Counter(test)

            # 중복된 값과 개수를 저장할 리스트
            data = []

            # Counter 객체에서 중복된 값과 개수를 추출하여 리스트에 저장
            for item, count in counter.items():
                if count > 1:
                    data.append({'값': item, '개수': count})

            # 데이터프레임 생성
            duplicated_df = pd.DataFrame(data)
            duplicated_df.sort_values(by="개수", ascending=False, inplace=True)
            duplicated_df.to_csv("result.csv", index=False, encoding='utf-8-sig')
            print(duplicated_df)

# Run the async main function
asyncio.run(main())


TimeoutError: 